In [83]:
# pip install autocorrect

In [84]:
# pip install joblib

In [1]:
import pandas as pd
import numpy as np
import seaborn as sb
from matplotlib import pyplot as plt
from datetime import datetime
import string

from transformers import AutoTokenizer, AutoModelForSequenceClassification
from nltk.tokenize import TweetTokenizer
from scipy.special import softmax

import nltk

import re
import contractions
import unidecode
from autocorrect import Speller

from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')

import torch
import os

[nltk_data] Downloading package stopwords to C:\Users\Krithika
[nltk_data]     JK\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to C:\Users\Krithika
[nltk_data]     JK\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [86]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to C:\Users\Krithika
[nltk_data]     JK\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [87]:
data = pd.read_csv('replyTweets.csv')

C:\Users\Krithika JK\AppData\Local\Temp\ipykernel_19652\908094907.py:1: DtypeWarning: Columns (6,9) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('replyTweets.csv')


# EDA

In [88]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 305783 entries, 0 to 305782
Data columns (total 17 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   id                   305779 non-null  float64
 1   createdAt            305780 non-null  object 
 2   fullName             305746 non-null  object 
 3   userName             305780 non-null  object 
 4   profileImage         305780 non-null  object 
 5   fullText             305780 non-null  object 
 6   replyTo              288845 non-null  object 
 7   lang                 305780 non-null  object 
 8   quoteCount           305780 non-null  float64
 9   retweetCount         305780 non-null  object 
 10  replyCount           305780 non-null  float64
 11  likeCount            305780 non-null  float64
 12  viewCount            120630 non-null  float64
 13  sentimentLabel1      1996 non-null    float64
 14  sentimentLabel2      1250 non-null    float64
 15  sentimentLabel3  

In [89]:
data.head(3)

,id,createdAt,fullName,userName,profileImage,fullText,replyTo,lang,quoteCount,retweetCount,replyCount,likeCount,viewCount,sentimentLabel1,sentimentLabel2,sentimentLabel3,sentimentLabelFinal
0,1.773239e+18,2024-03-28 06:43:19+00:00,Stella Patchouli,StellaPatch,https://pbs.twimg.com/profile_images/175397939...,@nytimes The curse on Musk iis working!\r\n🔥☔🐱🐱🤹💥,1772931578835939584.0,en,0.0,0,0.0,0.0,9.0,0.0,1.0,NaN,NaN
1,1.773174e+18,2024-03-28 02:21:41+00:00,Ceti Alpha V,RichGreenwood5,https://pbs.twimg.com/profile_images/122737855...,@elonmusk and @joebiden walk into a Chinese ba...,NaN,en,0.0,0,0.0,0.0,10.0,2.0,0.0,NaN,NaN
2,1.772932e+18,2024-03-27 10:20:02+00:00,The New York Times,nytimes,https://pbs.twimg.com/profile_images/109824457...,Elon Musk helped create China’s electric vehic...,NaN,en,5.0,20,52.0,98.0,113250.0,1.0,1.0,NaN,NaN


In [90]:
data.shape

(305783, 17)

In [91]:
data['lang'].value_counts()

en                                                    303962
tl                                                       506
ht                                                       369
fr                                                       222
es                                                       180
no                                                        71
et                                                        59
in                                                        55
de                                                        47
nl                                                        45
sv                                                        43
pt                                                        35
it                                                        34
qme                                                       26
da                                                        23
tr                                                        23
zxx                     

In [92]:
# Filter only eng tweets
data = data[data['lang']=='en']

In [93]:
data['lang'].value_counts()

en    303962
Name: lang, dtype: int64

# Data Pre-Processing

In [94]:
data['cleaned_tweet'] = data.fullText.str.lower()

In [95]:
# Function to remove URLs from text
def remove_urls(text):
    # Remove URLs starting with http or https
    text = re.sub(r'https?:\/\/\S+', '', text)
    # Remove URLs starting with www
    text = re.sub(r'www\.[a-z]?\.?(com)+|[a-z]+\.(com)', '', text)
    return text

# Apply the function to the 'cleaned_tweet' column
data['cleaned_tweet'] = data['cleaned_tweet'].apply(remove_urls)


In [96]:
data.cleaned_tweet = data.cleaned_tweet.apply(lambda x: re.sub(r'{link}', '', x))
data.cleaned_tweet = data.cleaned_tweet.apply(lambda x: re.sub(r"\[video\]", '', x))

In [97]:
# # Define a function to replace '@username' with '@user' except for '@elonmusk'
# def replace_username(text):
#     # Replace all occurrences of '@username' except '@elonmusk'
#     return re.sub(r'(@\w+)(?!elonmusk)', r'@user', text)

# # Apply the function to the 'tweet_text' column
# data['cleaned_tweet'] = data['cleaned_tweet'].apply(replace_username)


In [98]:
def remove_mentions(text):
    return re.sub(r'@\w+', '', text)
data['cleaned_tweet'] = data['cleaned_tweet'].apply(remove_mentions)

In [99]:
# Function to expand contractions
def expand_contractions(text):
    expanded_words = []
    for word in text.split():
        expanded_words.append(contractions.fix(word))
    return ' '.join(expanded_words)

# Apply the function to the 'fullText' column
data['cleaned_tweet'] = data['cleaned_tweet'].apply(expand_contractions)

In [100]:
# Remove accented characters from text using unidecode.
def remove_accented_characters(text):
    text = unidecode.unidecode(text)
    return text

data['cleaned_tweet'] = data['cleaned_tweet'].apply(remove_accented_characters)

In [101]:
import re

# Function to remove all non-letter characters from a string
def remove_non_letters(text):
    return re.sub(r'[^a-zA-Z\s]', '', text)

# Apply the function to the 'cleaned_tweet' column
data['cleaned_tweet'] = data['cleaned_tweet'].apply(remove_non_letters)

# Display the DataFrame with cleaned tweets
print(data)


                  id                  createdAt            fullName  \
0       1.773239e+18  2024-03-28 06:43:19+00:00    Stella Patchouli   
1       1.773174e+18  2024-03-28 02:21:41+00:00        Ceti Alpha V   
2       1.772932e+18  2024-03-27 10:20:02+00:00  The New York Times   
3       1.773129e+18  2024-03-27 23:25:20+00:00        Cha Ming Sul   
4       1.772992e+18  2024-03-27 14:22:04+00:00     Boston Strong 🍀   
...              ...                        ...                 ...   
305777  1.523967e+18  2022-05-10 10:02:01+00:00   Carlos 🌊🇺🇦🌲🏳️‍🌈🇵🇸   
305778  1.523966e+18  2022-05-10 09:58:03+00:00                 JYB   
305779  1.523966e+18  2022-05-10 09:57:59+00:00           Jannylove   
305780  1.523963e+18  2022-05-10 09:47:58+00:00          Ricky Hele   
305782  1.523962e+18  2022-05-10 09:45:33+00:00          Mr Neutron   

              userName                                       profileImage  \
0          StellaPatch  https://pbs.twimg.com/profile_images/175397939

In [102]:
# from joblib import Parallel, delayed
# from autocorrect import Speller
# from tqdm import tqdm

# def spell_correction_parallel(texts):
#     """
#     Perform spell correction on a list of texts using parallel processing with a progress bar.

#     Args:
#         texts (list): A list of input texts to be corrected.

#     Returns:
#         list: A list of corrected texts.
#     """
#     # Initialize the Speller object
#     spell = Speller()

#     # Define a function for spell correction
#     def correct_text(text):
#         return spell(text)

#     # Use parallel processing with tqdm progress bar
#     corrected_texts = Parallel(n_jobs=-1)(delayed(correct_text)(text) for text in tqdm(texts, desc="Spell Correction"))

#     return corrected_texts

# # Extract cleaned tweets from DataFrame column
# cleaned_tweets = data['cleaned_tweet'].tolist()

# # Perform spell correction on the cleaned tweets
# corrected_tweets = spell_correction_parallel(cleaned_tweets)

# # Replace the original cleaned tweets with the corrected ones
# data['cleaned_tweet'] = corrected_tweets

# # Save or use the DataFrame with corrected tweets as needed


In [103]:
import emoji

def convert_emojis_to_text(text):
    # Convert emojis to text
    text_with_emojis = emoji.demojize(text)
    return text_with_emojis

# Example usage
text_with_emojis = "I'm feeling 😊 today!"
text_without_emojis = convert_emojis_to_text(text_with_emojis)

# Apply to data
data['cleaned_tweet_vader'] = data['cleaned_tweet'].apply(convert_emojis_to_text)

print("Text with emojis:", text_with_emojis)
print("Text without emojis:", text_without_emojis)


Text with emojis: I'm feeling 😊 today!
Text without emojis: I'm feeling :smiling_face_with_smiling_eyes: today!


In [ ]:
#remove stop words
def remove_stopwords(text):
    stop_words = set(stopwords.words('english'))
    word_tokens = nltk.word_tokenize(text)
    filtered_text = [word for word in word_tokens if word.lower() not in stop_words]
    return ' '.join(filtered_text)

# Apply the remove_stopwords function to the 'cleaned_tweet' column
data['cleaned_tweet_vader'] = data['cleaned_tweet_vader'].apply(remove_stopwords)

In [ ]:
lemmatizer = nltk.stem.WordNetLemmatizer()
w_tokenizer = TweetTokenizer()
def lemmatize_text(text):
    return [(lemmatizer.lemmatize(w)) for w in \
                                     w_tokenizer.tokenize((text))]
data['tokens'] = data['cleaned_tweet'].apply(lemmatize_text)


In [ ]:
PUNCUATION_LIST = list(string.punctuation)
def remove_punctuation(word_list):
    """Remove punctuation tokens from a list of tokens"""
    return [w for w in word_list if w not in PUNCUATION_LIST]
data['tokens'] = data['tokens'].apply(remove_punctuation)

In [110]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 287188 entries, 0 to 305782
Data columns (total 19 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   id                   287187 non-null  float64
 1   createdAt            287188 non-null  object 
 2   fullName             287154 non-null  object 
 3   userName             287188 non-null  object 
 4   profileImage         287188 non-null  object 
 5   fullText             287188 non-null  object 
 6   replyTo              287188 non-null  object 
 7   lang                 287188 non-null  object 
 8   quoteCount           287188 non-null  float64
 9   retweetCount         287188 non-null  object 
 10  replyCount           287188 non-null  float64
 11  likeCount            287188 non-null  float64
 12  viewCount            103504 non-null  float64
 13  sentimentLabel1      1790 non-null    float64
 14  sentimentLabel2      1201 non-null    float64
 15  sentimentLabel3  

In [107]:
data.dropna(subset=['cleaned_tweet'], inplace=True)

In [48]:
data[['fullText', 'cleaned_tweet']].head(100)

,fullText,cleaned_tweet
0,@nytimes The curse on Musk iis working!\r\n🔥☔🐱🐱🤹💥,the curse on musk iis working!
1,@elonmusk and @joebiden walk into a Chinese ba...,and walk into a chinese bar. elon makes money ...
2,Elon Musk helped create China’s electric vehic...,elon musk helped create china's electric vehic...
3,Lies.\r\n\r\nMusk faced even greater challenge...,lies. musk faced even greater challenges from ...
4,@Sp4rksaflyin @nytimes These Electric Tesla ca...,these electric tesla cars cannot start in the ...
...,...,...
95,@global13news @guardiannews It’s a pretty repu...,"it is a pretty reputable source, sensor tower...."
96,@guardiannews Haha... Off course it has 😋\r\nA...,haha... off course it has and it is fake stat...
97,Please share this post with your friends who s...,please share this post with your friends who s...
98,@guardiannews No. This is erroneous.,no. this is erroneous.


In [50]:
data.to_csv('preprocesssed_data.csv', index=False)